In [ ]:
## Load CERES radiation anomaly dataset
ceres_rad_anom = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ceres_anom/ceres_anom_med_res.nc')

loCld_cre_anom = ceres_rad_anom.loCld_cre
highCld_cre_anom = ceres_rad_anom.highCld_cre
net_cre_anom = ceres_rad_anom.net_cre
clr_sky_anom = ceres_rad_anom.clr_sky
all_sky_anom = ceres_rad_anom.all_sky

In [ ]:
## Now, get low cloud cre anomaly for el nino/la nina years and split into seasons
def get_seas_cre(x):
    # Define empty numpy arrays to load data into
    count_en = range(0,len(el_nino_yrs))
    count_ln = range(0,len(la_nina_yrs))
    enso_JJA_back = []; enso_SON_back = []; enso_DJF = []; enso_MAM_for = []; enso_JJA_for = [];

    # For each el nino/la nina, get the mean for each season (JJA-1, SON-1, DJF, MAM+1, JJA+1) and append the empty arrays
    for i in count_en:
        en_start_year = el_nino_yrs[i][0]
        en_next_year  = el_nino_yrs[i][1]
        ln_start_year = la_nina_yrs[i][0]
        ln_next_year  = la_nina_yrs[i][1]

        JJA_back = (x.sel(time=slice(en_start_year+'-06',en_start_year+'-08')).mean(dim='time') -
                    x.sel(time=slice(ln_start_year+'-06',ln_start_year+'-08')).mean(dim='time'))

        SON_back = (x.sel(time=slice(en_start_year+'-09',en_start_year+'-11')).mean(dim='time') -
                    x.sel(time=slice(ln_start_year+'-09',ln_start_year+'-11')).mean(dim='time'))

        DJF      = (x.sel(time=slice(en_start_year+'-12',en_next_year+'-02')).mean(dim='time') -
                    x.sel(time=slice(ln_start_year+'-12',ln_next_year+'-02')).mean(dim='time'))

        MAM_for  = (x.sel(time=slice(en_next_year+'-03',en_next_year+'-05')).mean(dim='time') -
                    x.sel(time=slice(ln_next_year+'-03',ln_next_year+'-05')).mean(dim='time'))

        JJA_for  = (x.sel(time=slice(en_next_year+'-06',en_next_year+'-08')).mean(dim='time') -
                    x.sel(time=slice(ln_next_year+'-06',ln_next_year+'-08')).mean(dim='time'))

        enso_JJA_back.append(JJA_back)
        enso_SON_back.append(SON_back)
        enso_DJF.append(DJF)
        enso_MAM_for.append(MAM_for)
        enso_JJA_for.append(JJA_for)

    # Concatenate each numpy array with the seasonal mean for each el nino/la nina year, then average over all years
    JJA_back_enso_comp = xr.concat(enso_JJA_back,dim='count').assign_coords({'count':count_en}).mean(dim='count')
    SON_back_enso_comp = xr.concat(enso_SON_back,dim='count').assign_coords({'count':count_en}).mean(dim='count')
    DJF_enso_comp      = xr.concat(enso_DJF,dim='count').assign_coords({'count':count_en}).mean(dim='count')
    MAM_for_enso_comp  = xr.concat(enso_MAM_for,dim='count').assign_coords({'count':count_en}).mean(dim='count')
    JJA_for_enso_comp  = xr.concat(enso_JJA_for,dim='count').assign_coords({'count':count_en}).mean(dim='count')
    
    # Finally, put all DataArrays into one Dataset
    ceres_enso_composite = xr.Dataset(data_vars = {'JJA_back': JJA_back_enso_comp,
                                                   'SON_back': SON_back_enso_comp,
                                                   'DJF'     : DJF_enso_comp,
                                                   'MAM_for' : MAM_for_enso_comp,
                                                   'JJA_for' : JJA_for_enso_comp},
                                      coords    = {'latitude':new_lat_medRes,
                                                   'longitude':new_lon_medRes})
    
    # Get zonally averged CRE
    ceres_enso_composite_zonal = ceres_enso_composite.mean(dim='longitude').to_array() * np.cos(np.deg2rad(new_lat_medRes))
    ceres_enso_composite_zonal['variable'] = [0,1,2,3,4]
    
    # Get globally averged CRE
    ceres_enso_comp_avg = ceres_enso_composite.weighted(weight).mean(dim=('latitude','longitude')).round(3).to_array()
    ceres_enso_comp_avg['variable'] = [0,1,2,3,4]
    
    return ceres_enso_composite, ceres_enso_composite_zonal, ceres_enso_comp_avg

In [ ]:
loCld_enso_comp, loCld_enso_comp_zonal, loCld_enso_comp_glb = get_seas_cre(loCld_cre_anom)
highCld_enso_comp, highCld_enso_comp_zonal, highCld_enso_comp_glb = get_seas_cre(highCld_cre_anom)
net_cre_enso_comp, net_cre_enso_comp_zonal, net_cre_enso_comp_glb = get_seas_cre(net_cre_anom)
clr_sky_enso_comp, clr_sky_enso_comp_zonal, clr_sky_enso_comp_glb = get_seas_cre(clr_sky_anom)
all_sky_enso_comp, all_sky_enso_comp_zonal, all_sky_enso_comp_glb = get_seas_cre(all_sky_anom)

In [43]:
def get_ceres_cre(seas_index):
    # Set up subplots figure with PlateCarree projection
    fig, ax = plt.subplots(1, 5, figsize=(24,3), subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
    fig.tight_layout(w_pad=10)
    
    vmin = -6
    vmax = 6
    comp_title = ['All-Sky', 'Clear-Sky', 'Net CRE', 'Low Cloud CRE', 'High Cloud CRE']
    seas_title = ['JJA-1', 'SON-1', 'DJF', 'MAM+1', 'JJA+1']
    
    # Plot Trends
    cbar_data = all_sky_enso_comp.get(seas_index).plot(ax=ax[0],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)

    clr_sky_enso_comp.get(seas_index).plot(ax=ax[1],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)

    net_cre_enso_comp.get(seas_index).plot(ax=ax[2],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)

    loCld_enso_comp.get(seas_index).plot(ax=ax[3],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)

    highCld_enso_comp.get(seas_index).plot(ax=ax[4],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)

     # Give each plot gridlines, coastlines, and block out continents
    for i in np.arange(0,5,1):
        ax[i].gridlines()
        ax[i].coastlines()
        #ax[i].add_feature(cartopy.feature.LAND,zorder=100,facecolor='k',edgecolor='k')
        ax[i].set_aspect('auto')

        if seas_index == 'JJA_back':
            ax[i].set_title(comp_title[i], fontsize=20)
        elif seas_index == 'JJA_for':
            cbar_ax = fig.add_axes([0.01, -0.1, 0.98, 0.05])
            cbar = fig.colorbar(cbar_data, cax=cbar_ax, orientation='horizontal', extend='both')
            cbar.ax.tick_params(labelsize=20)
            cbar.set_label(label='$W/m^2/K$', size=20)

            ax[i].set_title(None)
        else:
            ax[i].set_title(None)

    fig.suptitle('Season = ' + seas_title[k], y=0.85, x=-0.01, rotation='vertical', fontsize=20)
    
    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Output/Plots/seasonal_composites/'
    fig.savefig(path + 'ceres_radiation_composites_' + seas_title[k] + 'seas_med_res.png', bbox_inches='tight')

In [ ]:
def get_zonal_ceres_cre():
    # Plot zonal mean vs season
    fig, ax = plt.subplots(1, 5, figsize=(36,9))
    fig.tight_layout(w_pad=9,h_pad=6)
    
    comp_title = ['All-Sky', 'Clear-Sky', 'Net CRE', 'Low Cloud CRE', 'High Cloud CRE']
    seas = ['JJA-1', 'SON-1', 'DJF', 'MAM+1', 'JJA+1']
    vmin = -2
    vmax = 2

    all_sky_enso_comp_zonal.transpose().plot(ax=ax[0],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '$W/m^2$'},vmin=vmin,vmax=vmax)
    
    clr_sky_enso_comp_zonal.transpose().plot(ax=ax[1],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '$W/m^2$'},vmin=vmin,vmax=vmax)
    
    net_cre_enso_comp_zonal.transpose().plot(ax=ax[2],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '$W/m^2$'},vmin=vmin,vmax=vmax)
    
    loCld_enso_comp_zonal.transpose().plot(ax=ax[3],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '$W/m^2$'},vmin=vmin,vmax=vmax)
    
    highCld_enso_comp_zonal.transpose().plot(ax=ax[4],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '$W/m^2$'},vmin=vmin,vmax=vmax)
    
    for i in range(0,5,1):
        ax[i].set_xticks([0,1,2,3,4])
        ax[i].set_xticklabels(seas)
        ax[i].set_xlabel('Season')
        ax[i].set_title('CERES El Nino Zonal Mean ' + comp_title[i] + ' Anomaly vs Season')
        
    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Output/Plots/seasonal_composites/'
    fig.savefig(path + 'zonal_mean_ceres_radiation_composites_seas.png', bbox_inches='tight')

In [ ]:
def get_gm_ceres_cre():
    fig, ax = plt.subplots(figsize=(18,12))
    seas = ['JJA-1', 'SON-1', 'DJF', 'MAM+1', 'JJA+1']
    xaxis = [0,1,2,3,4]
    
    ax.plot(xaxis,all_sky_enso_comp_glb, 'purple', marker='o', label='ALL SKY')
    ax.plot(xaxis,clr_sky_enso_comp_glb, 'green', marker='o', label='CLEAR SKY')
    ax.plot(xaxis,net_cre_enso_comp_glb, 'black', marker='o', label='NET CRE')
    ax.plot(xaxis,loCld_enso_comp_glb, 'red', marker='o', label='LOW CLOUD CRE')
    ax.plot(xaxis,highCld_enso_comp_glb, 'blue', marker='o', label='HIGH CLOUD CRE')
    
    ax.set_xticks(xaxis)
    ax.set_xticklabels(seas)
    ax.set_yticks(np.linspace(-0.75,0.3,10))
    ax.axhline(y=0.0, color='k', linestyle='dotted')
    ax.axvline(x=2, color='k', linestyle='dotted')
    ax.set_xlabel('Season')
    ax.set_ylabel('$W/m^2$')
    ax.legend()
        
    ax.set_title('CERES El Nino Global Mean Radiation Anomalies vs Season')
    
    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Output/Plots/seasonal_composites/'
    fig.savefig(path + 'glob_mean_ceres_radiation_composites_seas.png', bbox_inches='tight')